## Calculando PPL de las transcripciones

Para leer sobre la perplejidad consultar los siguientes links: 
https://huggingface.co/spaces/evaluate-metric/perplexity 
https://huggingface.co/docs/transformers/perplexity

In [1]:
import warnings
warnings.filterwarnings('ignore')
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [2]:
model_id = "openai-community/gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [3]:
with open('transcriptions/transcription_large.txt', 'r', encoding="utf-8") as file:
    transcription = file.read()
    print(len(transcription))
    print(transcription)

5606
 Buenas tardes, el día de hoy vamos a presentar el trabajo final de estadística vallisiana por Nicolás Gil Lugo y mi persona Laura Lucía Domínguez. Como contenido tenemos la introducción, problema de investigación, formas analíticas de las distribuciones, estimaciones e indicadores, conclusiones y la bibliografía. Muy buenas tardes, como introducción entonces tenemos que en este trabajo se aborda un problema específico desde el enfoque vallisiano para la estimación de un parámetro que para nosotros es de interés. El punto de vista vallisiano considera un parámetro desconocido como una característica con respecto a la cual puede expresarse un grado de credibilidad la cual puede ser modificada mediante la información muestral. El problema de investigación. Es que un médico oncólogo desea analizar los niveles de tiroglobulina en un grupo de 35 indebidos con cáncer de tiroides después de un año de ser sometidos a radioterapia con yodo. Y este tratamiento es exitoso si los niveles de t

In [4]:
encodings = tokenizer("\n\n".join(transcription), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (15886 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
# base
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl

 93%|█████████▎| 28/30 [03:37<00:15,  7.78s/it]


tensor(3.8597)

In [5]:
# large
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl

 94%|█████████▍| 30/32 [04:05<00:16,  8.19s/it]


tensor(3.7032)